# Parallelism in Python

### John Kirkham

# The Problem

* Typical threading models are hard for (new) users to understand
* Easy to run into difficult to debug scenarios (e.g. deadlocking, race conditions, etc.)
* Implementation often becomes tied to a certain scale (e.g. multithreaded code -> cluster parallelized code)
* How could this be done better?

# Task-based parallelism

* Describe the pieces of the computation
* Relate these pieces to each other
* Use a scheduler to perform the computation

# Common implementations

* Dask
* ipyparallel
* Luigi